In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas(desc='My bar!')
from collections import defaultdict

/cm/shared/apps/spack/gpu/opt/spack/linux-centos8-skylake_avx512/gcc-8.3.1/anaconda3-2020.11-bsn4npoxyw7jzz7fajncek3bvdoaa5wv/lib/python3.8/site-packages/tqdm/std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
genes = pd.read_csv("Homo_sapiens.GRCh38.108.gtf", sep = "\t", comment="#", header=None)
genes[0] = "chr" + genes[0].astype(str)


def find_genes(row):
    gene_df = genes[(genes[0] == row['CHROM']) & (genes[3] <= row['POS']) & 
                    (genes[4] >= row['POS'])]
    if len(gene_df) == 0:
        return None
    gene_df_info = gene_df[8].str.split(";")
    gene_set = set()
    for info in list(gene_df_info):
        for field in info:
            if "gene_name" in field:
                name = field.strip().replace("gene_name ","").replace('"','')
                gene_set.add(name)
    if len(gene_set) == 0:
        return None
                
    return ",".join(list(gene_set))

/cm/shared/apps/spack/gpu/opt/spack/linux-centos8-skylake_avx512/gcc-8.3.1/anaconda3-2020.11-bsn4npoxyw7jzz7fajncek3bvdoaa5wv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
all_expansions = pd.DataFrame(columns = ['CHROM', 'POS', 'PERIOD', 'MOTIF', 'outlier_threshold', 'AFR_freq',
       'NON_AFR_freq'])
for i in range(1,23):
    if i == 9:
        continue
    expansions = pd.read_csv(f"{i}_expansions.csv")
    all_expansions = pd.concat([expansions, all_expansions])
    
    
all_expansions['gene'] = all_expansions.apply(lambda row: find_genes(row), axis = 1)

In [5]:
pedigree = pd.read_csv("/expanse/projects/gymreklab/helia/TR_1000G/1000G.ped", delim_whitespace=True)
pedigree = pedigree[['SampleID','Superpopulation', 'Population']]

samp_to_subpop = pd.Series(pedigree[pedigree['Superpopulation'] == "AFR"].Population.values,index=pedigree[pedigree['Superpopulation'] == "AFR"].SampleID).to_dict()
samp_to_pop = pd.Series(pedigree.Superpopulation.values,index=pedigree.SampleID).to_dict()
H3Africa_names = pd.read_csv("/expanse/projects/gymreklab/helia/H3Africa/names/H3A_Baylor_sample_country.txt", header=None, delim_whitespace=True)

for index,row in H3Africa_names.iterrows():
    samp_to_pop[row[0]] = "H3Africa"
    samp_to_subpop[row[0]] = row[1]
    
african_pop = set(list(samp_to_subpop.values()))

In [6]:
def find_freqs(row):
    chrom = row['CHROM'].replace("chr", "")
    header = []
    with open(f"/expanse/projects/gymreklab/helia/ensembl/experiments/charact/diff_dist/diff/{chrom}_diff.txt") as f:
        for line in f:
            if line.startswith("chr"):
                    f.close()
                    break
            header.append(line.strip())

    addr=f"/expanse/projects/gymreklab/helia/ensembl/experiments/charact/diff_dist/diff/{chrom}_diff.txt"

    pos = row['POS']
    x=!grep -m 1 -w $pos $addr
    gbs = x[0].split("\t")[4:]
    dict_ = dict(zip(header, gbs))
    
    for x in dict_:
        dict_[x] = dict_[x].split("/")
        modified_gbs = []
        for s in dict_[x]:
            if s != "." and s != "":
                modified_gbs.append(int(s)/ row['PERIOD'])
                
        dict_[x] = modified_gbs
        
        
        
    pop_freqs = {}
    for pop in ['AFR', 'H3Africa']:
        gbs = []
        for samp in dict_:
            if samp in samp_to_pop:
                if samp_to_pop[samp] == pop:
                    gbs.extend(dict_[samp])
                
        if len(gbs) == 0:
            pop_freqs[pop] = -1
        else:
            above_threshold = round(len([x for x in gbs if x not in [".",""] \
                       and x > row['outlier_threshold']]) / len([x for x in gbs if x not in [".",""]]),3)
            pop_freqs[pop] = above_threshold
                
                
            
    
    
    
    return pop_freqs

all_expansions[['AFR_freqs', 
                        'H3Africa_freqs']] = all_expansions.apply(lambda row: find_freqs(row), 
                                                                         axis = 1, result_type="expand")
all_expansions

,CHROM,POS,PERIOD,MOTIF,outlier_threshold,AFR_freq,NON_AFR_freq,gene,AFR_freqs,H3Africa_freqs
0,chr22,17976536,2,AC,6.0,0.027958,0.000000,MICAL3,0.028,0.029
1,chr22,20888311,2,AC,8.0,0.077357,0.000866,SNAP29,0.071,0.093
2,chr22,22722990,2,AG,9.0,0.042406,0.000000,None,0.049,0.039
3,chr22,24042817,2,AC,16.0,0.032510,0.000650,CABIN1,0.036,0.024
4,chr22,24496373,2,AC,7.0,0.018623,0.000435,UPB1,0.017,0.023
...,...,...,...,...,...,...,...,...,...,...
173,chr1,247419145,2,AT,4.5,0.027386,0.000655,NLRP3,0.030,0.021
174,chr1,247919144,2,AC,5.0,0.081789,0.001732,None,0.077,0.093
175,chr1,248249150,2,AT,8.0,0.055197,0.001083,None,0.058,0.047
176,chr1,248684754,2,AT,6.0,0.072522,0.002599,None,0.074,0.069


In [22]:
all_expansions_both = all_expansions[(all_expansions['AFR_freq'] < 0.01) | 
                                     (all_expansions['AFR_freqs'] > 0.01) & 
                                     (all_expansions['H3Africa_freqs'] > 0.01)]
all_expansions_both['outlier_threshold'] = all_expansions_both['outlier_threshold'].round(1)
all_expansions_both['NON_AFR_freq'] = all_expansions_both['NON_AFR_freq'].round(3)
all_expansions_both['AFR_freq'] = all_expansions_both['AFR_freq'].round(3)
all_expansions_both.columns = ['CHROM','POS','PERIOD','MOTIF','OUTLIER_THRESH',
                               'ALL_AFR_FREQ','NON_AFR_FREQ','GENE','1KG_AFR_FREQ','H3Africa_FREQ']
all_expansions_both = all_expansions_both[all_expansions_both['OUTLIER_THRESH'] > 10]
#all_expansions_both.sort_values('OUTLIER_THRESH', ascending=False).to_csv('Large_Expansions.csv', index = False)

<ipython-input-22-48b6c8015c9b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_expansions_both['outlier_threshold'] = all_expansions_both['outlier_threshold'].round(1)
<ipython-input-22-48b6c8015c9b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_expansions_both['NON_AFR_freq'] = all_expansions_both['NON_AFR_freq'].round(3)
<ipython-input-22-48b6c8015c9b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

In [57]:
#### Compare with Ibra's results on ExpansionHunter Denovo and STRetch ####


EH = pd.read_csv("/expanse/projects/gymreklab/helia/ensembl/experiments/charact/diff_dist/STRetch-EHdn/EHdn/loci_detected_by_EHdn.csv")
stretch = pd.read_csv("/expanse/projects/gymreklab/helia/ensembl/experiments/charact/diff_dist/STRetch-EHdn/STRetch/TRs_detected_by_STRetch.csv")
stretch.columns = ['chr', 'start', 'end', 'motif', 'gene', 'dist']

def support(row, method):
    if method == "EH":
        compare_df = EH.copy()
    if method == "stretch":
        compare_df = stretch.copy()

    df = compare_df[(compare_df['chr'] == row['CHROM']) & 
                    (compare_df['start'] >= row['POS'] - 1000 ) & (compare_df['start'] < row['POS'] + 1000)]
    if len(df) == 0:
        return "N"
    if len(df) > 1:
        cnt = 0
        for index, row1 in df.iterrows():
            if row1['motif'] == row['MOTIF']:
                cnt += 1
        if cnt == 1:
            return "Y"
        else:
            print(f"Multiple records at locus {row['POS']}")
            display(df)
            return "U"
    if len(df) == 1 and str(df['motif'].iloc[0]) == str(row['MOTIF']):
        return "Y"
    if len(df) == 1 and str(df['motif'].iloc[0]) != str(row['MOTIF']):
        print(f"Different motif at locus {row['POS']}")
        return "U"
    else:
        display(df)


all_expansions_both['EH_supported'] = all_expansions_both.apply(lambda x: support(x, "EH"), axis = 1)
all_expansions_both['STRetch_supported'] = all_expansions_both.apply(lambda x: support(x, "stretch"), axis = 1)
all_expansions_both

Different motif at locus 92185591


,CHROM,POS,PERIOD,MOTIF,OUTLIER_THRESH,ALL_AFR_FREQ,NON_AFR_FREQ,GENE,1KG_AFR_FREQ,H3Africa_FREQ,EH_supported,STRetch_supported
3,chr22,24042817,2,AC,16.0,0.033,0.001,CABIN1,0.036,0.024,N,Y
5,chr22,24757660,2,AC,16.0,0.013,0.001,PIWIL3,0.013,0.011,N,N
6,chr22,26667333,2,AC,11.0,0.081,0.001,MIAT,0.078,0.089,N,N
16,chr22,39686600,2,AT,11.0,0.003,0.037,CACNA1I,0.003,0.001,N,N
18,chr22,41958483,2,AG,12.0,0.070,0.002,SMIM45,0.076,0.055,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...
148,chr1,227297941,2,AC,12.0,0.028,0.001,CDC42BPA,0.027,0.029,N,N
149,chr1,227297968,2,AC,12.0,0.031,0.001,CDC42BPA,0.030,0.032,N,N
159,chr1,235882040,2,AC,12.0,0.027,0.000,LYST,0.027,0.029,N,N
167,chr1,244553603,2,AC,13.0,0.044,0.001,CATSPERE,0.040,0.056,N,N


In [51]:
stretch.columns

Index(['chr', 'start', 'end', 'repeatunit', 'gene', 'dist'], dtype='object')